<a href="https://colab.research.google.com/github/acponce2023/a2/blob/main/filtering_process_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este cuaderno se encuentra una adaptación del proceso de filtrado para que podáis trabajar con él y ver su funcionamiento más en detalle.

Hay algunos ficheros que tendréis que poner en las carpetas que corresponda, por ejemplo todos los ficheros de frecuencias.


In [ ]:
!pip install spacy lingua-language-detector


In [19]:
!tar -xvf freq.tar
!tar -xvf extract.tar

freq/fre/
freq/fre/fre-1gram.txt
freq/spa/spa-1gram.txt
freq/ger/
freq/spa/
freq/ita/ita-2gram.txt
freq/ita/ita-1gram.txt
freq/ger/ger-2gram.txt
freq/ger/ger-1gram.txt
freq/eng/
freq/
freq/eng/eng-2gram.txt
freq/ita/
freq/eng/eng-1gram.txt
freq/fre/fre-2gram.txt
freq/spa/spa-2gram.txt
extract/eng/occupational_therapy/
extract/spa/occupational_therapy/
extract/fre/occupational_therapy/
extract/spa/
extract/spa/occupational_therapy/terms.txt
extract/fre/occupational_therapy/terms.txt
extract/ger/occupational_therapy/terms.txt
extract/eng/
extract/ger/occupational_therapy/
extract/ita/
extract/eng/occupational_therapy/terms.txt
extract/ita/occupational_therapy/terms.txt
extract/fre/
extract/ita/occupational_therapy/
extract/ger/
extract/


In [ ]:
!spacy download en_core_web_sm
!spacy download de_core_news_sm
!spacy download fr_core_news_sm
!spacy download it_core_news_sm
!spacy download es_core_news_sm

In [21]:
import time
import traceback
import uuid
import os
import spacy
import json

import lingua

# Spacy

spacy_models = json.loads('''{
	"bul": {"sm": null, "lg": null},
	"hrv": {"sm": "hr_core_news_sm", "lg": "hr_core_news_lg"},
	"cze": {"sm": null, "lg": null},
	"dan": {"sm": "da_core_news_sm", "lg": "da_core_news_trf"},
	"dut": {"sm": "nl_core_news_sm", "lg": "nl_core_news_lg"},
	"eng": {"sm": "en_core_web_sm", "lg": "en_core_web_trf"},
	"est": {"sm": null, "lg": null},
	"fin": {"sm": "fi_core_news_sm", "lg": "fi_core_news_lg"},
	"fre": {"sm": "fr_core_news_sm", "lg": "fr_dep_news_trf"},
	"ger": {"sm": "de_core_news_sm", "lg": "de_dep_news_trf"},
	"gre": {"sm": "el_core_news_sm", "lg": "el_core_news_lg"},
	"hun": {"sm": null, "lg": null},
	"gle": {"sm": null, "lg": null},
	"ita": {"sm": "it_core_news_sm", "lg": "it_core_news_lg"},
	"lav": {"sm": null, "lg": null},
	"lit": {"sm": "lt_core_news_sm", "lg": "lt_core_news_lg"},
	"mlt": {"sm": null, "lg": null},
	"pol": {"sm": "pl_core_news_sm", "lg": "pl_core_news_lg"},
	"por": {"sm": "pt_core_news_sm", "lg": "pt_core_news_lg"},
	"rum": {"sm": "ro_core_news_sm", "lg": "ro_core_news_lg"},
	"rus": {"sm": "ru_core_news_sm", "lg": "ru_core_news_lg"},
	"slo": {"sm": null, "lg": null},
	"slv": {"sm": null, "lg": null},
	"spa": {"sm": "es_core_news_sm", "lg": "es_dep_news_trf"},
	"swe": {"sm": null, "lg": null}
}''')


freq_list = {"spa": {}, "ger": {}, "fre": {}, "eng": {}, "ita": {}}

freq_list["spa"]["1-gram"] = open("./freq/spa/spa-1gram.txt").read().split("\n")
freq_list["ger"]["1-gram"] = open("./freq/ger/ger-1gram.txt").read().split("\n")
freq_list["fre"]["1-gram"] = open("./freq/fre/fre-1gram.txt").read().split("\n")
freq_list["eng"]["1-gram"] = open("./freq/eng/eng-1gram.txt").read().split("\n")
freq_list["ita"]["1-gram"] = open("./freq/ita/ita-1gram.txt").read().split("\n")

freq_list["spa"]["2-gram"] = open("./freq/spa/spa-2gram.txt").read().split("\n")
freq_list["ger"]["2-gram"] = open("./freq/ger/ger-2gram.txt").read().split("\n")
freq_list["fre"]["2-gram"] = open("./freq/fre/fre-2gram.txt").read().split("\n")
freq_list["eng"]["2-gram"] = open("./freq/eng/eng-2gram.txt").read().split("\n")
freq_list["ita"]["2-gram"] = open("./freq/ita/ita-2gram.txt").read().split("\n")
## --->
print("Num. palabras unicas en spa: ",len(freq_list["spa"]["1-gram"]))
print("Num. 2 palabras asociadas en spa: ", len(freq_list["spa"]["2-gram"]))
#print(freq_list["spa"]["1-gram"])
#print(freq_list["spa"]["2-gram"])
print("Num. palabras unicas en eng: ", len(freq_list["eng"]["1-gram"]))
print("Num. 2 palabras asociadas en eng: ", len(freq_list["eng"]["2-gram"]))
#print(freq_list["eng"]["1-gram"])
#print(freq_list["eng"]["2-gram"])
print("Num. palabras unicas en ger: ", len(freq_list["ger"]["1-gram"]))
print("Num. 2 palabras asociadas en ger: ", len(freq_list["ger"]["2-gram"]))
#print(freq_list["ger"]["1-gram"])
#print(freq_list["ger"]["2-gram"])
print("Num. palabras unicas en fre: ", len(freq_list["fre"]["1-gram"]))
print("Num. 2 palabras asociadas en fre: ", len(freq_list["fre"]["2-gram"]))
#print(freq_list["fre"]["1-gram"])
#print(freq_list["fre"]["2-gram"])
print("Num. palabras unicas en ita: ", len(freq_list["ita"]["1-gram"]))
print("Num. 2 palabras asociadas en ita: ", len(freq_list["ita"]["2-gram"]))
#print(freq_list["ita"]["1-gram"])
#print(freq_list["ita"]["2-gram"])
## --->
lingua_langs = {
		"eng": lingua.Language.ENGLISH,
		"spa": lingua.Language.SPANISH,
		"ita": lingua.Language.ITALIAN,
		"ger": lingua.Language.GERMAN,
		"fre": lingua.Language.FRENCH
	}

langs_used = [v for k,v in lingua_langs.items()]
## --->
print(langs_used)
## --->
lang_detector = lingua.LanguageDetectorBuilder.from_languages(*langs_used).build()
## --->
print (lang_detector)
## --->

Num. palabras unicas en spa:  50001
Num. 2 palabras asociadas en spa:  100001
Num. palabras unicas en eng:  3136241
Num. 2 palabras asociadas en eng:  5866953
Num. palabras unicas en ger:  50001
Num. 2 palabras asociadas en ger:  100001
Num. palabras unicas en fre:  50001
Num. 2 palabras asociadas en fre:  100001
Num. palabras unicas en ita:  748540
Num. 2 palabras asociadas en ita:  77739
[Language.ENGLISH, Language.SPANISH, Language.ITALIAN, Language.GERMAN, Language.FRENCH]


In [22]:
example = open("./extract/spa/occupational_therapy/terms.txt").readlines()
## -->
print("Total de terminos a filtrar spa: ", len(example))


Total de terminos a filtrar spa:  7581


In [ ]:
## -->
terms = filter_terms(example, "spa")
## -->


In [27]:
## -->
print("Total terminos filtrados spa: ", len(terms))
#print("Terminos filtrados", terms)
## -->

Total terminos filtrados spa:  1225


In [28]:
terms = lemmatize_terms(terms, "spa")
## -->
print("Total terminos lematizados spa: ", len(terms))
#print("Terminos lematizados: ", terms)
## -->
# Matriz Confusion
# Definimos el listado inicial como A y el listado final como B, los valores de la matriz los podemos razonar de la siguiente forma:
# A :terminos a filtrar  SPA ; B: Términos filtrados y lematizados
#TP: la intersección de A y B
#FP: los valores de A no contenidos en B
#FN: los valores de B no contenidos en A
#TN: ¿?

# TP      FN
# FP      TN
#----------->
# 1.197   0
# 6.654   0

print("PRECISION = TP /(TP + FP) =" , 1197 / 7581)

Total terminos lematizados spa:  1197
PRECISION = TP /(TP + FP) = 0.15789473684210525


In [29]:
example = open("./extract/ger/occupational_therapy/terms.txt").readlines()
## -->
print("Total de terminos a fltrar ger: ", len(example))

Total de terminos a fltrar ger:  4001


In [ ]:
## -->
terms = filter_terms(example, "ger")
## -->

In [31]:
print("Total terminos filtrados ger", len(terms))
#print("Terminos filtrados", terms)
## -->

Total terminos filtrados ger 1210


In [32]:
terms = lemmatize_terms(terms, "ger")
## -->
print("Total terminos lematizados ger: ", len(terms))
#print("Terminos lematizados: ", terms)
## -->
# Matriz Confusion
# Definimos el listado inicial como A y el listado final como B, los valores de la matriz los podemos razonar de la siguiente forma:
# A :terminos a filtrar  GER ; B: Términos filtrados y lematizados
#TP: la intersección de A y B
#FP: los valores de A no contenidos en B
#FN: los valores de B no contenidos en A
#TN: ¿?

# TP      FN
# FP      TN
#----------->
# 1.157   0
# 2.844   0

print("PRECISION = TP /(TP + FP) =" , 1157 / 4001)

Total terminos lematizados ger:  1157
PRECISION = TP /(TP + FP) = 0.2891777055736066


In [33]:
example = open("./extract/fre/occupational_therapy/terms.txt").readlines()
## -->
print("Total de terminos a filtrar fre: ", len(example))

Total de terminos a filtrar fre:  5742


In [ ]:
terms = filter_terms(example, "fre")

In [35]:
## -->
print("Total terminos filtrados fre", len(terms))
#print("Terminos filtrados", terms)
## -->

Total terminos filtrados fre 1015


In [36]:
## -->
terms = lemmatize_terms(terms, "fre")
## -->
print("Total terminos lematizados fre: ", len(terms))
#print("Terminos lematizados: ", terms)
## -->
# Matriz Confusion
# Definimos el listado inicial como A y el listado final como B, los valores de la matriz los podemos razonar de la siguiente forma:
# A :terminos a filtrar  FRE ; B: Términos filtrados y lematizados
#TP: la intersección de A y B
#FP: los valores de A no contenidos en B
#FN: los valores de B no contenidos en A
#TN: ¿?

# TP      FN
# FP      TN
#----------->
#   981    0
# 4.761    0

print("PRECISION = TP /(TP + FP) =" , 981 / 5742)

Total terminos lematizados fre:  981
PRECISION = TP /(TP + FP) = 0.17084639498432602


In [37]:
example = open("./extract/eng/occupational_therapy/terms.txt").readlines()
## -->
print("Total de terminos a filtrar eng: ", len(example))

Total de terminos a filtrar eng:  4902


In [ ]:
terms = filter_terms(example, "eng")

In [39]:
## -->
print("Total terminos filtrados eng", len(terms))
#print("Terminos filtrados", terms)
## -->

Total terminos filtrados eng 1263


In [40]:
## -->
terms = lemmatize_terms(terms, "eng")
## -->
print("Total terminos lematizados eng: ", len(terms))
#print("Terminos lematizados: ", terms)
## -->
# Matriz Confusion
# Definimos el listado inicial como A y el listado final como B, los valores de la matriz los podemos razonar de la siguiente forma:
# A :terminos a filtrar  ENG ; B: Términos filtrados y lematizados
#TP: la intersección de A y B
#FP: los valores de A no contenidos en B
#FN: los valores de B no contenidos en A
#TN: ¿?

# TP      FN
# FP      TN
#----------->
# 1159   0
# 4116   0

print("PRECISION = TP /(TP + FP) =" , 1159/ 4902)

Total terminos lematizados eng:  1159
PRECISION = TP /(TP + FP) = 0.2364341085271318


In [41]:
example = open("./extract/ita/occupational_therapy/terms.txt").readlines()
## -->
print("Total de terminos a filtrar ita: ", len(example))

Total de terminos a filtrar ita:  2709


In [ ]:
terms = filter_terms(example, "ita")

In [43]:
## -->
print("Total terminos filtrados ita", len(terms))
#print("Terminos filtrados", terms)
## -->

Total terminos filtrados ita 393


In [45]:
## -->
terms = lemmatize_terms(terms, "ita")
## -->
print("Total terminos lematizados ita: ", len(terms))
#print("Terminos lematizados: ", terms)
## -->
# Matriz Confusion
# Definimos el listado inicial como A y el listado final como B, los valores de la matriz los podemos razonar de la siguiente forma:
# A :terminos a filtrar  ITA ; B: Términos filtrados y lematizados
#TP: la intersección de A y B
#FP: los valores de A no contenidos en B
#FN: los valores de B no contenidos en A
#TN: ¿?

# TP      FN
# FP      TN
#----------->
# 388   0
# 2.321   0

print("PRECISION = TP /(TP + FP) =" , 388/ 2709)

Total terminos lematizados ita:  387
PRECISION = TP /(TP + FP) = 0.14322628276116647


In [24]:
def filter_terms(lines, lang):

	terms = {}
	filter_deep_1g = 50001
	filter_deep_2g = 1000000
	#filter_deep_1g = 3200000
	#filter_deep_2g = 6000000

	dict_1g = {}
	dict_2g = {}


	if (lang in freq_list) and ("1-gram" in freq_list[lang]):

		lower_list = [t.lower() for t in freq_list[lang]["1-gram"][:filter_deep_1g]]

		dict_1g = dict(zip(lower_list, range(len(lower_list))))
## --->
#	print (dict_1g)
## --->
	if (lang in freq_list) and ("2-gram" in freq_list[lang]):

		lower_list = [t.lower() for t in freq_list[lang]["2-gram"][:filter_deep_2g]]

		dict_2g = dict(zip(lower_list, range(len(lower_list))))
## --->
#	print(dict_2g)
## --->

	for term in lines:

		freq, term = term.replace("\n", "").split("\t")

		term = term.replace("-", " ").replace("  ", " ")

		if (lang in freq_list) and ("1-gram" in freq_list[lang]) and (term.lower() in dict_1g):

			print("Excluding", term, "(too freq 1-gram)")


		elif (lang in freq_list) and ("2-gram" in freq_list[lang]) and (term.lower() in dict_2g):

			print("Excluding", term, "(too freq 2-gram)")


		elif any(len(word) < 4 for word in term.split(" ")):

			print("Excluding", term, "(too short)")


		elif not term.replace(" ", "").replace("'", "").replace("-","").isalpha() or term.replace(" ", "").startswith("-") or term.replace(" ", "").endswith("-"):

			print("Excluding", term, "(strange symbols)")

		else:

			print("Adding", term)

			terms[term] = {"f": freq}


	# Las diferencias de capitalizacion se resuelven optando por la version mas habitual

	for term, obj in terms.copy().items():

		if term.lower() != term and term.lower() in terms:

			if int(terms[term.lower()]["f"]) > int(terms[term]["f"]):

				terms.pop(term)

				print("Excluding", term, "(duplicated and less frequent capitalization)")

			else:

				terms.pop(term.lower())

		print("Excluding", term.lower(), "(duplicated and less frequent capitalization)")


	valid_NE = ["EVENT", "FAC", "ORG", "WORK_OF_ART"]

	pipe = spacy.load(spacy_models[lang]["sm"])

	for term, obj in terms.copy().items():

		doc = pipe(term)

		for token in doc.ents:

			print("Found NE: ", token.text, token.label_)

			if not (token.label_ in valid_NE) and term in terms:

				terms.pop(term)


	for term, obj in terms.copy().items():

		detected = lang_detector.detect_language_of(term)

		print(term, detected)

		if lang in lingua_langs and detected != lingua_langs[lang]:

			terms.pop(term)


	return terms

In [25]:
def lemmatize_terms(terms, lang):

	lemmatized_terms = {}

	pipe = spacy.load(spacy_models[lang]["sm"])

	term_list_old = list(terms.keys())

	for term in term_list_old:

		doc = pipe(term)

		full_token = []

		for token in doc:

			full_token.append(token.lemma_)

		lemma = " ".join(full_token)

		if term in terms:

			old_f = terms[term]

			if lemma in lemmatized_terms:

				current_f = lemmatized_terms[lemma]

				new_f = current_f["f"] + old_f["f"] # Varias palabras convergen en una raíz

				lemmatized_terms[lemma] = {"f": new_f}

			else:

				lemmatized_terms[lemma] = {"f": old_f["f"]}

	# Se reaplica filtrado a las palabras luego de filtrarlas, esto estaría
  # mejor hacerlo de otra forma, hay código repetido

	filter_deep_1g = 500001
	filter_deep_2g = 1000000
	#filter_deep_1g = 3200000
	#filter_deep_2g = 6000000

	dict_1g = {}
	dict_2g = {}

	if (lang in freq_list) and ("1-gram" in freq_list[lang]):

		lower_list = [t.lower() for t in freq_list[lang]["1-gram"][:filter_deep_1g]]

		dict_1g = dict(zip(lower_list, range(len(lower_list))))

	if (lang in freq_list) and ("2-gram" in freq_list[lang]):

		lower_list = [t.lower() for t in freq_list[lang]["2-gram"][:filter_deep_2g]]

		dict_2g = dict(zip(lower_list, range(len(lower_list))))


	for term in lemmatized_terms.copy().keys():

    # Solo se extá empleando en "eng" porque la lematización en otras cambia
    # también otras flexiones y es algo a evitar.

		if lang == "eng":

			if (term.lower() in dict_1g):

#				print("Excluding", term, "(too freq 1-gram) lemma")

				lemmatized_terms.pop(term)

			elif (term.lower() in dict_2g):

	#			print("Excluding", term, "(too freq 2-gram) lemma")

				lemmatized_terms.pop(term)

	return lemmatized_terms
